In [ ]:
# !pip install xgboost

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier 
from sklearn import metrics
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')

In [5]:
df_train= pd.read_csv('/content/drive/MyDrive/Regex Team 3/Team Project - 3/creditcard_stratified.csv')
df_test= pd.read_csv('/content/drive/MyDrive/Regex Team 3/Team Project - 3/test.csv')
df_train.head()

,id,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class,kfold
0,0,171306.0,2.168118,-1.731966,-0.101423,-1.516228,-1.762327,0.107145,-1.803075,0.087421,-0.617820,1.430516,-1.132033,-0.548229,0.989762,-1.007329,0.142895,-0.110690,0.292566,0.222226,-0.358004,-0.298250,-0.126096,0.104613,0.337718,0.583060,-0.529252,-0.273834,0.060092,-0.017113,52.00,0,4.0
1,1,62395.0,-0.514037,0.953548,1.639592,-0.149570,0.061661,-0.210378,0.545380,0.203165,-0.678399,-0.191564,1.707444,0.803937,-0.082876,0.374202,0.381713,0.042487,-0.293247,-0.374638,-0.132844,0.076307,-0.133674,-0.327980,0.025932,0.205276,-0.322368,0.070640,0.277572,0.109626,5.85,0,4.0
2,2,43463.0,-0.671869,-0.399551,1.649636,-2.278123,-0.484760,-1.164731,0.155846,-0.030425,1.440249,-1.829221,-0.583126,0.890476,0.592870,-0.457628,0.409242,-0.194237,-0.481948,0.095075,-0.063800,0.116415,0.210261,0.594446,0.184314,0.430021,-0.644314,-0.342088,0.171030,0.207153,70.00,0,3.0
3,3,157918.0,2.084298,0.103235,-1.613616,0.485971,0.264824,-1.411857,0.598777,-0.504130,0.496313,-0.078829,-0.784751,0.783872,0.424063,0.359565,-0.384585,-0.843043,-0.117172,-0.719843,0.226709,-0.239673,0.002001,0.293965,0.029691,0.090537,0.383237,-0.224996,-0.031261,-0.065743,1.00,0,1.0
4,4,67461.0,1.286755,-1.194764,1.503195,-0.353358,-1.869471,0.605151,-1.749865,0.461911,0.668069,0.467496,0.024983,-0.197757,-1.296228,-0.960619,-1.907411,0.911665,0.595563,-0.603601,1.410409,-0.027345,0.155647,0.710105,-0.129878,0.051088,0.449721,-0.038945,0.073214,0.011353,5.80,0,1.0


In [6]:
useful_features = [c for c in df_train.columns if c not in ("Class", "kfold")]
df_test = df_test[useful_features]


final_test_predictions = [] #to store final test predictions
final_valid_predictions = {} # to store final validation predictions
scores = [] #to store the scores

# using for loop for 5 folds
for fold in range(5):
    
    # Making all values which are not equal to present fold as training set
    xtrain =  df_train[df_train.kfold != fold]
    # Making all values which are equal to present fold as Validation set
    xvalid = df_train[df_train.kfold == fold]
    # cMaking copy of test set
    xtest = df_test.copy()
    
    #Creating keys for the dictionary containing valid predictions
    valid_ids = xvalid['id'].values.tolist()

    #creating ytrain and yvalid
    ytrain = xtrain.Class
    yvalid = xvalid.Class
    
    #creating xtrain and xvalid
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    
#     #ordinal encoder
#     ordinal_encoder = preprocessing.OrdinalEncoder()
#     xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
#     xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
#     xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
#     param = {
#         "objective": "regression",
#         "metric": "rmse",
#         "verbosity": -1,
#         "boosting_type": "gbdt",
#         "n_estimators": 10000,
#         "early_stopping_round": 300,
#         "device": "gpu",
#         "gpu_platform_id": 0,
#         "gpu_device_id": 0,
#     }
    
#     param2 = {
#         'lambda_l1': 0.00472279780583036, 
#         'lambda_l2': 2.9095205689488508e-05, 
#         'num_leaves': 158, 
#         'feature_fraction': 0.7386878356648194, 
#         'bagging_fraction': 0.8459744550725283, 
#         'bagging_freq': 2, 
#         'max_depth': 2, 
#         'max_bin': 249, 
#         'learning_rate': 0.044738463593017294,
#         'min_child_samples': 13
#     }
#     param.update(param2)
    
    model = XGBClassifier(n_estimators  = 200, random_state= 7, device = "gpu", gpu_platform_id = 0, gpu_device_id = 0)
    model.fit(xtrain, 
            ytrain,
            eval_set=[(xvalid, yvalid)],
            early_stopping_rounds=200,
            verbose=500,eval_metric= 'auc'
        )

    preds_valid = model.predict_proba(xvalid)[:, 1]
    test_preds = model.predict_proba(xtest)[:, 1]    
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    roc_auc_score = metrics.roc_auc_score(yvalid, preds_valid)
    print(f'Fold {fold} AUC: ', roc_auc_score)
    scores.append(roc_auc_score)
    
    
print(np.mean(scores), np.std(scores))
# final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
# final_valid_predictions.columns = ["id", "pred_LGB"]
# final_valid_predictions.to_csv("train_pred_LGB.csv", index=False)

# sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
# sample_submission.columns = ["id", "pred_LGB"]
# sample_submission.to_csv("test_pred_LGB.csv", index=False)

[0]	validation_0-auc:0.954855
Will train until validation_0-auc hasn't improved in 200 rounds.
[199]	validation_0-auc:0.99489
Fold 0 AUC:  0.9950743761516816
[0]	validation_0-auc:0.914562
Will train until validation_0-auc hasn't improved in 200 rounds.
[199]	validation_0-auc:0.983154
Fold 1 AUC:  0.9904519208735458
[0]	validation_0-auc:0.897505
Will train until validation_0-auc hasn't improved in 200 rounds.
[199]	validation_0-auc:0.973519
Fold 2 AUC:  0.9738099234919095
[0]	validation_0-auc:0.915461
Will train until validation_0-auc hasn't improved in 200 rounds.
[199]	validation_0-auc:0.981169
Fold 3 AUC:  0.989036952012267
[0]	validation_0-auc:0.881789
Will train until validation_0-auc hasn't improved in 200 rounds.
[199]	validation_0-auc:0.963862
Fold 4 AUC:  0.9720130837313136
0.9840772512521436 0.009350229492100817
